### Scraping Comments https://www.blackwhitereadallover.com/

In [15]:
# Libraries

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from random import randint
from time import sleep 


In [16]:
# Setting up the Chrome webdriver

options = webdriver.ChromeOptions()
options.add_argument('--headless') #uncomment to not render driven browser during scrapping

options.binary_location = "/usr/bin/google-chrome"
chrome_driver_binary = "/usr/local/bin/chromedriver"
#options.binary_location = "C:/Program Files (x86)/Google/Chrome/Application/chrome.exe"
#chrome_driver_binary = "C:/Users/Alek/chromedriver"
driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)



/home/ravisolter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


### Scraping the Archives

In [24]:
# Create a list of archive URLs (Feb, 2007 - March, 2020)

archive_periods = []

for j in range(2007,2021):
    for k in range(1,13):
        single_period = 'https://www.blackwhitereadallover.com/archives/'+str(j)+'/'+str(k)
        archive_periods.append(single_period)
        ;
        
# Subsetting out some periods for which there are no articles
active_archives = archive_periods[1:len(archive_periods)-8]

#len(active_archives) #159

#print(active_archives[100])
        

https://www.blackwhitereadallover.com/archives/2015/6


In [25]:
## Collecting the article URLS from each archive page, storing in 'archive_links' list

archive_links = []

for u in range(98,len(archive_periods)):
    print(u)
    
    sleep(randint(4, 7))  # sleep between 1 and 3 seconds
    
    driver.get(active_archives[u])
    
    
    # Checking to see if 'Load More' button exists (for months with>30 articles) and if so clicking it
    button = driver.find_elements_by_xpath("//button[contains(text(),'Load More')]")
    while len(button) > 0 :
        button[0].click()  
        sleep(7)
        button = driver.find_elements_by_xpath("//button[contains(text(),'Load More')]")

 
    
    # Retrieving number of articles from title 
    # OLD archive_title = driver.find_element_by_xpath('/html/body/div[1]/div[2]/h1')
    archive_title = driver.find_element_by_xpath("//h1[contains(text(),'Archives for')]")

    #print(archive_title.text)
    
    start = archive_title.text.find('(')+1
    start = archive_title.text[start:]
    articles = int(start.replace(')',''))
    #print(articles)
    
    
    # Retrieving the article links themselves
    for q in range(0,articles):
        
        article_link = driver.find_elements_by_css_selector('h2.c-entry-box--compact__title>a')
        article_link = article_link[q].get_attribute('href')
        
    
        archive_links.append(article_link) 
           
    ;



98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159


IndexError: list index out of range

In [19]:
# Binding together the resulting .txts. There are 6k articles in total

links1 = pd.read_csv('archive_links_1_78.txt',header=None)
links2 = pd.read_csv('archive_links_79_97.txt',header=None)
links3 = pd.read_csv('archive_links_98_end.txt',header=None)

links12 = links1.append(links2, ignore_index = True) 
archive_links_full = links12.append(links3, ignore_index = True)

archive_links_full.head()

archive_links_full.shape



(6035, 1)

In [20]:
# Turning it back into a list
archive_links_full = archive_links_full[0].to_list()

### Logging in to BRWAO

In [21]:
# Logging in to BWRAO to have access to comment recs

driver.get("https://auth.voxmedia.com/login?return_to=https://www.blackwhitereadallover.com/")

element = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CLASS_NAME, "p-text-input"))
    )

username = driver.find_element_by_xpath('//*[@id="login[username]"]')
username.clear()
username.send_keys("rsolter")


password = driver.find_element_by_xpath('//*[@id="login[password]"]')
password.clear()
password.send_keys("pass123!")


button = driver.find_element_by_xpath('//*[@id="auth"]/div/form/fieldset[3]/input')
button.click()

In [22]:
# Have selenium wait until the BWRAO logo loads (therefore, has logged in properly)
element = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CLASS_NAME, "c-global-header__logo-large"))
    )

#### Scraping relevant info from individual pages

In [23]:
## Gathering blog title, author, publish date and comment list and comment count from a the loop of 'archive_links'

comments_df = pd.DataFrame()
for k in range(1,len(archive_links_full)):
#for k in range(1,20):

    
    print(str(k)+' of '+str(len(archive_links_full)))
    
    driver.get(archive_links_full[k])

    ## Grab article title, author

    blog_title = driver.find_element_by_xpath('//*[@id="content"]/article/div[1]/div[1]/div[2]/h1')
    article_author = driver.find_element_by_xpath('//*[@id="content"]/article/div[1]/div[1]/div[3]/span[1]/span[1]/a/span')
    article_publish_date = driver.find_element_by_xpath('//*[@id="content"]/article/div[1]/div[1]/div[3]/span[1]/span[2]/time')


    #Check for comments, and load, then run everything
    test = len(driver.find_elements_by_xpath('//*[@id="comments"]'))
    if test > 0:
        sleep(2)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        element = WebDriverWait(driver, 100).until(
                EC.presence_of_element_located((By.CLASS_NAME, "c-comments__list")))

        #Find number of comments

        n_comments = driver.find_element_by_xpath('//*[@id="comments"]/div[1]/span/span')
        n_comments = int(n_comments.text)


        # Looping through all comments to gather comment title, comment body, poster, and number of recs


    

        for i in range(1,n_comments+1):
    
            xpath = '//*[@id="comments"]/div[2]/div['+str(i)+']'
     
            comment_title = driver.find_element_by_xpath(xpath + '/div[1]')
            comment_body = driver.find_element_by_xpath(xpath + '/div[2]')
    
            # Generally, comment meta data is stored in the 3rd div except when commentors have a signature
            # in which case the meta data is in the 4th div
            
            #recs aren't weren't put in till later, and I was too lazy to find out when
            #so I just check if they exist first
            test = driver.find_elements_by_xpath(xpath+'/div[3]/span[2]/button[2]/span[2]')
            test2 = driver.find_elements_by_xpath(xpath+'/div[4]/span[2]/button[2]/span[2]')
            

            if driver.find_elements_by_xpath(xpath + '/div[4]'):
                meta = driver.find_element_by_xpath(xpath+'/div[4]/span[1]')
                meta = meta.text.rsplit(' on ',1)
                post_date = meta[1]
                poster = meta[0].split(' by ')[1]
    
                if len(test2) > 0:
                # Some comments have no recommendations
                    if driver.find_elements_by_xpath(xpath + '/div[4]/span[2]/button[2]/span[2]'):
                        recs = driver.find_element_by_xpath(xpath + '/div[4]/span[2]/button[2]/span[2]')
                        recs = int(recs.text.replace('(','').replace(')',''))
                    else:
                        recs = 0
            else:
                meta = driver.find_element_by_xpath(xpath+'/div[3]/span[1]')
                meta = meta.text.rsplit(' on ',1)
                post_date = meta[1]
                poster = meta[0].split(' by ')[1]
        
                if len(test) >0:
                # Some comments have no recommendations
                    if driver.find_elements_by_xpath(xpath + '/div[3]/span[2]/button[2]/span[2]'):
                        recs = driver.find_element_by_xpath(xpath + '/div[3]/span[2]/button[2]/span[2]')
                        recs = int(recs.text.replace('(','').replace(')',''))
                    else:
                        recs = 0
                        
            if len(test) == 0 & len(test2) == 0:
                recs = ""


            comment = pd.DataFrame([blog_title.text,article_author.text,article_publish_date.text,comment_title.text,comment_body.text,poster,post_date.split('|')[0],
                            post_date.split('|')[1],recs])
            comments_df = pd.concat([comments_df,comment.T])
    sleep(4)
    ;
   

1 of 6035
2 of 6035
3 of 6035
4 of 6035
5 of 6035
6 of 6035
7 of 6035
8 of 6035
9 of 6035
10 of 6035
11 of 6035
12 of 6035
13 of 6035
14 of 6035
15 of 6035
16 of 6035
17 of 6035
18 of 6035
19 of 6035
20 of 6035
21 of 6035
22 of 6035
23 of 6035
24 of 6035
25 of 6035
26 of 6035
27 of 6035
28 of 6035
29 of 6035
30 of 6035
31 of 6035
32 of 6035
33 of 6035
34 of 6035
35 of 6035
36 of 6035
37 of 6035
38 of 6035
39 of 6035
40 of 6035
41 of 6035
42 of 6035
43 of 6035
44 of 6035
45 of 6035
46 of 6035
47 of 6035
48 of 6035
49 of 6035
50 of 6035
51 of 6035
52 of 6035
53 of 6035
54 of 6035
55 of 6035
56 of 6035
57 of 6035
58 of 6035
59 of 6035
60 of 6035
61 of 6035
62 of 6035
63 of 6035
64 of 6035
65 of 6035
66 of 6035
67 of 6035
68 of 6035
69 of 6035
70 of 6035
71 of 6035
72 of 6035
73 of 6035
74 of 6035
75 of 6035
76 of 6035
77 of 6035
78 of 6035
79 of 6035
80 of 6035
81 of 6035
82 of 6035
83 of 6035
84 of 6035
85 of 6035
86 of 6035
87 of 6035
88 of 6035
89 of 6035
90 of 6035
91 of 6035
92 of 60

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="content"]/article/div[1]/div[1]/div[2]/h1"}
  (Session info: headless chrome=80.0.3987.163)


### Aggregating into data frame, exporting

In [25]:
comments_df.columns = ['Article Title','Article Author','Article Publish Date','Comment Title','Comment Body','Comment Poster','Comment Date','Comment Time','Comment Recs']
comments_df.head()

print(k)

545


In [26]:
comments_df.to_csv('Comments_1_545.csv', index=False)


In [40]:
driver.quit()